# Convert local pixels to geographical coordinates

This notebook facilitates the conversion of local pixel coordinates within 50% segments contained in a polygon to geographical coordinates (latitude, longitude). This transformation is achieved through a Transformer derived from an object representing a single-band acquisition. 

In [1]:
import json
import pandas as pd 
import rasterio as rio
from pyproj import Transformer

In [2]:
# Transform the crs of our file in order to convert
path = "./tif_data/"
image_path = path + "aquisition01_band02.tif"
src = rio.open(image_path)
data = src.read(1)
in_epsg = src.crs
out_epsg = "EPSG:4326"
transformer = Transformer.from_crs(in_epsg, out_epsg)

In [3]:
def get_lat_long(pixel0, pixel1):
    """
    This function use the transformer set up above to 
    transform point between the two coordinates systems.

    Args:
        pixel0 (int): coordinate of the pixel in the image
        pixel1 (int): coordinate of the pixel in our image

    Returns:
        lat: latitude coordinate of the pixel
        long: longitude coordinate of the pixel
    """
    coords = src.xy(pixel0, pixel1)
    lat, long = transformer.transform(coords[0], coords[1])
    return lat, long

In [4]:
import unittest

class TestGetLatLongFunction(unittest.TestCase):

    def test_get_lat_long(self):
        # Sample data for testing
        pixel0 = 100
        pixel1 = 200

        result = get_lat_long(pixel0, pixel1)

        # Expected result
        expected_result = (11.362317328663496, -3.874558554420653)  

        # Assertion
        self.assertEqual(result, expected_result, "Function output does not match expected result.")

unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestGetLatLongFunction))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [5]:
df = pd.read_csv("./data/Inference_results_water.txt", sep=",")

results_by_segment = {}

for index, row in df.iterrows():
    # extracting relevant information from the row
    segment_id = row['segment_id']
    class_id = row['class_id']

    # extracting pixel coordinates from the DataFrame
    perimeter_pixels_list = eval(row['Perimeter Pixels'])
    pixels_list = eval(row['pixels'])
    alphas = eval(row['heatmap'])

    segment_coordinates = []
    perimeter_coordinates = []

    # applying the function to each perimeter pixel and storing the results for 'Perimeter Pixels'
    for perimeter_pixel in perimeter_pixels_list:
        lat, long = get_lat_long(perimeter_pixel[0], perimeter_pixel[1])
        coordinates = {
            "latitude": lat,
            "longitude": long,
            "intensity": 10
        }
        perimeter_coordinates.append(coordinates)
        
    # applying the function to each pixel and storing the results for 'Pixels'
    for pixel in pixels_list:
        lat, long = get_lat_long(pixel[0], pixel[1])
        coordinates = {
            "latitude": lat,
            "longitude": long,
            "intensity": 2
        }
        segment_coordinates.append(coordinates)

    results_by_segment[segment_id] = {
        "segment_id": segment_id,
        "class_id": class_id,
        "pixel_coordinates": segment_coordinates,
        "perimeter_pixel_coordinates": perimeter_coordinates,
        "alphas": alphas 
    }

# write the results to a JSON file
with open("final_water_segment_lat_long.json", "w") as json_file:
    json.dump(list(results_by_segment.values()), json_file, indent=2)

print("Results have been written to 'final_water_segment_lat_long.json'.")


Results have been written to 'final_water_segment_lat_long.json'.
